### data preparation

In [4]:
import numpy as np
import cv2
import os




##### KNN #######

def distance(v1,v2):
    #eucledian
    return np.sqrt(((v1-v2)**2).sum())

def knn(train,test,k=5) :
    dist = []
    
    
    for i in range (train.shape[0]) :
        ## get the vector and label
        ix = train[i, :-1]
        iy = train[i,-1]
        
        #compute distance from test point
        d = distance(test , ix)
        dist.append([d,iy])
        
        ## sort based on distance and get top k
        
        dk = sorted(dist,key=lambda x:x[0])[:k]
        
        #retrieve only the labels
        labels = np.array(dk)[:,-1]
        
        #get frequency of each label
        output = np.unique(labels,return_counts=True)
        
        #Find max freq and respective label
        index = np.argmax(output[1])
        return output[0][index]

######## KNN  End #####



#init camera
cap = cv2.VideoCapture(0)

#face detection
face_cascade = cv2.CascadeClassifier('/home/ankur/Documents/haarcascade_frontalface_default.xml')


skip = 0
data_path = '/home/ankur/Desktop/Face Data/'

face_data = []    ##x-data
labels = []      ##y- data

class_id = 0  ## labels for given file
names = {} ##mapping b/w id - name




### data preparation

for fx in os.listdir(data_path) : 
    
    ## in command type 'listdir' inside face data folder to get all files info
    
    if fx.endswith('.npy'):
        
        ## creating mapping btw class_id and name
        names[class_id] = fx[:-4]    ##providing name but slicing '.npy'
        print("Loaded"+fx)
        data_item = np.load(data_path+fx,allow_pickle=True)
        face_data.append(data_item)
        
        ## create labels for class
        
        target = class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
    
        ## at first array of zero is mult by A.npy
        ## for second,array of 1 is mult by B.npy
        
        ## Now concatinate all lists in single list
        
face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0)

print(face_dataset.shape)
print(face_labels.shape)


## making x and y data in single matrix so we can use train data theory

trainset = np.concatenate((face_dataset,face_labels),axis = 1)



######## TESTING  #########

while True:
    ret,frame = cap.read()
    if ret==False:
        continue
        
    faces = face_cascade.detectMultiScale(frame,1.3,5)
    
    for face in faces :
        x,y,w,h = face
        
        ## get face (crop)
        offset = 10   ## providing 10 pixel of margin to all four sides
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_sectione = cv2.resize(face_section,(100,100))
        
        
    #### call knn ###    
        
        
        ## predicted label(out)
        out = knn(trainset,face_section.flatten())
        
        
        
        ## dislpay name and rectangle around it
        pred_name = names[Sint(out)]
        cv2.putText(frame,pred_name,(x,y-10),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        
        
    cv2.imshow("Faces",frame)
    
    key = cv2.waitKey(1) & 0xFF 
        
    if key == ord('q'):
        break  
        
        
cap.release()
cv2.destroyAllWindows()

LoadedAnkur.npy
(14, 1)
(14,)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)